In [1]:
import json
import time
import datetime
import requests 
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/

In [2]:
CONFIG_FILE = Path('.')/Path('0-setup')/Path('fec_config.yaml')
with open(CONFIG_FILE, 'r') as f:
    config = yaml.load(f)

In [3]:
api_key = config['fec']['api_key']

# Interfacing with API 
Goal: Input: Candidate, Year, District:


Output: Overall Financials, Financials in Given District 


Desired Format: .csv


May have to use: node.js, puppeter, AWS, Python Threading (? Later on ?)

### Construct Candidate_id DataFrame A

In [4]:
#Retrieve Comprehensive Candidate Dictionary

i = 1
candidate_dict = []
while True:
    candidate_id = requests.get('https://api.open.fec.gov/v1/candidates/?sort=name\
&has_raised_funds=true\
&min_first_file_date=1980\
&per_page=100\
&api_key=' + api_key + '\
&page='+str(i))
    if candidate_id.json()['results'] ==  []:
        break
    else: 
        results = candidate_id.json()['results']
        candidate_dict = candidate_dict +  results
        i += 1
print('Done')
print(i)

Done
188


In [5]:
df_general_cand = pd.DataFrame(candidate_dict)
df_general_cand.head()

,active_through,candidate_id,candidate_status,cycles,district,district_number,election_districts,election_years,federal_funds_flag,first_file_date,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2018,S2UT00229,C,"[2012, 2014, 2016, 2018]",00,0.0,"[00, 00]","[2012, 2018]",False,2012-02-08,...,Open seat,2018-04-23,2018-04-23,2018-08-01T00:13:22,"AALDERS, TIMOTHY NOEL",S,Senate,CON,CONSTITUTION PARTY,UT
1,2012,H2CA01110,P,"[2012, 2014, 2016]",01,1.0,[01],[2012],False,2012-02-22,...,Challenger,2012-02-22,2012-02-22,2013-04-26T09:04:30,"AANESTAD, SAMUEL",H,House,REP,REPUBLICAN PARTY,CA
2,2018,H8CO06237,C,[2018],06,6.0,[06],[2018],False,2017-04-26,...,Challenger,2017-04-26,2017-04-26,2017-08-01T20:57:28,"AARESTAD, DAVID",H,House,DEM,DEMOCRATIC PARTY,CO
3,2008,P80002926,N,"[2006, 2008, 2010, 2012, 2014, 2016]",00,0.0,[00],[2008],False,2005-10-12,...,Open seat,2007-03-13,2007-03-13,2016-11-17T06:10:48,"AARON, LAURA DAVIS",P,President,DEM,DEMOCRATIC PARTY,US
4,2000,H0MA01024,P,"[2000, 2002, 2004]",01,1.0,[01],[2000],False,2000-02-02,...,Challenger,2000-02-02,2000-02-02,2002-04-12T00:00:00,"ABAIR, PETER JON",H,House,REP,REPUBLICAN PARTY,MA


In [6]:
df_candidate_id = df_general_cand[['name', 
                                   'candidate_id', 
                                   'state',
                                   'incumbent_challenge_full', 
                                   'party_full',
                                   'office_full',
                                   'cycles',
                                   'election_years',
                                   'last_file_date',
                                   'load_date']]
number_candidates = df_candidate_id.shape[0]
print('Number of Candidates Collected: '+  str(number_candidates))
df_candidate_id.head()
#df_candidate_id.name.to_csv('candidate_names.csv')

Number of Candidates Collected: 18617


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [7]:
#Test
df_candidate_id[df_candidate_id.name == 'AALDERS, TIMOTHY NOEL']


,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22


In [31]:
#To csv to skip the run times 
df_candidate_id.to_csv('candidate_ids.csv')

### Construct Committee_id DataFrame B

In [9]:
#Retrieve Comprehensive Committee Dictionary

i = 1
committee_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/committees/\
?per_page=100&sort_null_only=false&sort=name\
&sort_hide_null=false\
&page='+str(i) + '\
&min_first_file_date=2010\
&api_key=' + api_key

    committee_id = requests.get(url)
    if committee_id.json()['results'] ==  []:
        break
    else: 
        results = committee_id.json()['results']
        committee_dict = committee_dict +  results
        i += 1
print('Done')
print(i)

Done
199


In [10]:
df_general_comm = pd.DataFrame(committee_dict)
df_general_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,organization_type,organization_type_full,party,party_full,state,treasurer_name
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,None,None,W,WRITE-IN,MA,"TEIXEIRA, RASHARNDA SHELINE MISS"
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",None,None,None,None,DC,ANGELIQUE HARRIS
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,None,None,None,None,CA,RANDI M WILLIS
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-11-02,11/06/18 VICTORY FUND,None,None,None,None,MS,"KEETON, LYNETTE"
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-21,1106 FUND,None,None,None,None,DC,"MIELNIK, MEGAN"


In [11]:
df_committee_id = df_general_comm.drop(columns=['name', 'organization_type', 'organization_type_full', 'party', 'party_full', 'treasurer_name'])
empty = []
df_committee_id = df_committee_id.loc[df_committee_id.candidate_ids.map(len) > 0].reset_index()
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC


In [12]:
# Add column for metric to weight the effect of contribution 
# on each candidate represented by committee

# Simple approach, to be updated at a later point

number_candidates = np.array([len(x) for x in df_committee_id.candidate_ids])**float(-1)
df_committee_id['candidate_donor_weight'] = number_candidates

In [13]:
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC,1.000


In [32]:
df_committee_id.to_csv('committee_id.csv')

## Retrieve Committee Donors 

### Takes too long, extraneous information, inefficient approach

In [ ]:
#    "per_page": 100,
#    "pages": 507467,
#    "count": 50746611

i = 0
last_index = 0
donor_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/schedules/schedule_a/?\
per_page=100\
&last_index={}\
&sort_null_only=false\
&is_individual=true\
&min_date=2010\
&two_year_transaction_period=2018\
&sort_hide_null=false\
&api_key={}'.format(last_index, api_key) 
    
    donor = requests.get(url)
    
    
    try:
        print(donor.json())
    except json.decoder.JSONDecodeError:
        print("Encountered JSON Decoder Error")
        break
    
    if (donor.status_code == 429):
        print('429 Response Received. Sleep for an hour')
        print(datetime.datetime.now())
        print(i)
        print(last_index)
        time.sleep(3610) #Sleep for an hour & 10 Seconds before continuing.
        print('Starting up again')
        candidate_committee = requests.get(url)
        if candidate_committee.status_code == 429:
            break;
    
    print(last_index)
    print(i)
    donor = donor.json()
    donor_results = donor['results']
    
    if donor_results == []:
        break
    else: 
        donor_dict = donor_dict + donor_results 
        last_index_contribution_date = [donor['pagination']['last_indexes']['last_index'],
                                       donor['pagination']['last_indexes']['last_contribution_receipt_date']]
        last_index = last_index_contribution_date[0]
        i += 1
        
    
donor_dict

In [83]:
donor_df = pd.DataFrame(donor_dict)
columns = ['contribution_receipt_amount', 
      'contribution_receipt_date',
      'receipt_type_desc',
      'entity_type_desc',
      'contributor_city',
      'contributor_state',
      'contributor_name',
      'party_full',
      'candidate_id',
      'committee_id',
      'committee',
      'line_number_label',
      'fec_election_type_desc']
donor_df = donor_df[columns]


## Better Approach

In [15]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
    #print(donor_cand_size.status_code)
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    #print(donor_cand_state.status_code)
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [16]:
#Test
a = cand_donor_state('P80001571', 2016)
cand_donor_state('P80001571', 2016)


,candidate_id,cycle,state,state_full,total
0,P80001571,2016,AA,Armed Forces Americas,476.75
1,P80001571,2016,AE,Armed Forces Europe,7579.01
2,P80001571,2016,AK,Alaska,368816.42
3,P80001571,2016,AL,Alabama,1674759.75
4,P80001571,2016,AP,Armed Forces Pacific,2435.77
5,P80001571,2016,AR,Arkansas,739125.42
6,P80001571,2016,AS,American Samoa,4684.00
7,P80001571,2016,AZ,Arizona,2771362.70
8,P80001571,2016,CA,California,12082510.57
9,P80001571,2016,CO,Colorado,2275468.86


In [17]:
# Presidential Election, Likeley has donations from all entities within the United States. 
states = a.state_full
#states = np.append(states, 'Armed Forces Americas')
states.head()

0    Armed Forces Americas
1      Armed Forces Europe
2                   Alaska
3                  Alabama
4     Armed Forces Pacific
Name: state_full, dtype: object

In [18]:
cand_donor_size('P80001571', 2018)


,candidate_id,cycle,size,total
0,P80001571,2018,0,19373283.84
1,P80001571,2018,200,3405918.70
2,P80001571,2018,500,1179909.12
3,P80001571,2018,1000,1141544.03
4,P80001571,2018,2000,5518142.30


In [19]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    #print(donor_comm_size.status_code)
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    #print(donor_comm_state.status_code)
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [20]:
#Test 
comm_donor_state('C00624759')

,committee_id,count,cycle,state,state_full,total
0,C00624759,61,2016,IN,Indiana,75000.0
1,C00624759,1,2016,MI,Michigan,2500.0


In [21]:
comm_donor_size('C00624759')

,committee_id,count,cycle,size,total
0,C00624759,21.0,2016,200,5300.0
1,C00624759,24.0,2016,1000,25500.0
2,C00624759,NaN,2016,0,675.0
3,C00624759,12.0,2016,2000,44200.0
4,C00624759,5.0,2016,500,2500.0


## Compile Comprehensive DataFrame of Donors per Candidate

### 1. For Donations made directly to a candidate


### 2. For Donations made to a committee that has a list of candidates that they support.

In [22]:
df_candidate_id.head()

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,UT,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,CA,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,CO,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,US,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,MA,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [23]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

In [63]:
# Input: Candidate Name, Election Type, Date-time of election (Use Pandas Date time Format)
# 1. Will incorporate a funcation call to 
# 2. to retrieve the fraction of committee donations for that candidate

# by_size/by_candidate
#  -0    $200 and under Level 1 
#  -200  $200.01 - $499.99 Level 2 
#  -500  $500 - $999.99 Level 3 
#  -1000 $1000 - $1999.99 Level 4
#  -2000 $2000 + Level 5

a = cand_donor_state('P80001571', 2016)
states = a.state_full


# Returns rows, 1 row per cycle with all features for one candidate. 
def candidate_donor_df(candidate_name, states, df_candidate_id):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    
    for cand_id in cand_ids:
        print(cand_id)
    
        cycles = cand_row.cycles.values[0]


        

        #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
        
        print(cycles)
        for cycle in cycles:
            df_size = cand_donor_size(cand_id, cycle)
            df_state = cand_donor_state(cand_id, cycle)
            if df_size.values.size == 0 or df_state.values.size == 0:
                continue

            donor_states = df_state.state_full

            #Build row of information (donor_states)

            # Idea
            # Create dictionary with states as keys and initialized with 0s as values. 
            initialize_array = np.zeros(len(states))
            state_dict = dict(zip(states, initialize_array))

            for ds in donor_states:            
                # Overwrite select values with information from df_state
                state_dict[ds] = df_state[df_state.state_full == ds].total.values[0]


            # Add size features to state_dict 


            level_array = np.zeros(5)
            level_dict = dict(zip(donation_levels, level_array))

            for level in df_size['size']:
                if level == 0:
                    donation_level = 'Donation Level 1'
                if level == 200:
                    donation_level = 'Donation Level 2'
                if level == 500:
                    donation_level = 'Donation Level 3'
                if level == 1000:
                    donation_level = 'Donation Level 4'
                if level == 2000:
                    donation_level = 'Donation Level 5'
                level_dict[donation_level] = df_size[df_size['size'] == level].total.values[0]


            cand_info_dict = {'name': candidate_name, 
                              'cand_id': cand_id, 
                              'state': cand_row.state.values[0],
                              'cycle': cycle,
                              'incumbent': cand_row.incumbent_challenge_full.values[0],
                              'office_full': cand_row.office_full.values[0],
                              'party': cand_row.party_full.values[0]}
            state_dict = merge_two_dicts(cand_info_dict, state_dict)
            state_dict = merge_two_dicts(state_dict, level_dict)
            #return state_dict


            #return state_dict.values()
            df.loc[j] = list(state_dict.values()) # An array of the row information
            j += 1
    return df

                         
                         
        

In [64]:
#b = candidate_donor_df('PELOSI, NANCY', states)
#b = candidate_donor_df('TRUMP, DONALD J', states, df_candidate_id)
b = candidate_donor_df('HELLER, DEAN', states, df_candidate_id)

b

H6NV02164
[2006, 2008, 2010, 2012, 2014, 2016, 2018]
S2NV00183
[2006, 2008, 2010, 2012, 2014, 2016, 2018]


,name,cand_id,state,cycle,incumbent,office_full,party,Armed Forces Americas,Armed Forces Europe,Alaska,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5
0,"HELLER, DEAN",H6NV02164,NV,2006,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.00,2252.00,68136.98,93719.30,257432.35,734148.00
1,"HELLER, DEAN",H6NV02164,NV,2008,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.00,23790.00,25302.00,82883.09,196782.52,586850.00
2,"HELLER, DEAN",H6NV02164,NV,2010,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.00,76271.01,30830.00,69050.00,137586.02,419550.00
3,"HELLER, DEAN",H6NV02164,NV,2012,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.00,556.02,3114.00,10400.00,34346.65,71300.00
4,"HELLER, DEAN",S2NV00183,NV,2012,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,12000.0,7000.00,59583.33,7550.0,3342.86,828267.78,266980.27,543719.42,1006881.40,2627682.78
5,"HELLER, DEAN",S2NV00183,NV,2014,Challenger,House,REPUBLICAN PARTY,0.0,0.0,0.0,...,0.0,0.00,0.00,0.0,0.00,15996.00,14150.00,55988.00,113950.00,206400.00
6,"HELLER, DEAN",S2NV00183,NV,2016,Challenger,House,REPUBLICAN PARTY,0.0,0.0,1000.0,...,4000.0,0.00,0.00,0.0,0.00,3895.00,15131.12,115661.48,194977.80,533800.00
7,"HELLER, DEAN",S2NV00183,NV,2018,Challenger,House,REPUBLICAN PARTY,0.0,0.0,450.0,...,3380.0,38658.33,27900.00,8999.0,0.00,1209270.55,134652.21,390485.86,939859.48,3745516.67


# Do Committe's Have additional donations which are not represented by the Candidate Donations. For Now assume that the donations to committees which then fund candidates are accounted for in the above. 

In [26]:
len(df_committee_id.committee_id)

8850

In [27]:
# Run this, build committee donor extraction algorithm that adds to the other function. 
# Run for all candidates. Try and find the election results first tho.
df_committee_id.head()


def reformat_committee_df(df):
    df_temp = df
    df_comm = pd.DataFrame(columns = df_temp.columns)
    i = 1
    for list_cand_id in df_temp.candidate_ids:
        print(i)
        for cand_id in list_cand_id:
            df_temp.iloc[i-1:i,:] = df_temp.iloc[i-1:i,:].drop(columns='candidate_ids')
            df_temp.iloc[i-1:i,:]['candidate_ids'] = cand_id
            df_comm = pd.concat([df_comm, df_temp.iloc[i-1:i,:]])
        i+=1
        if i == 100:
            break
    return df_comm



In [28]:
df_comm = reformat_committee_df(df_committee_id)
df_comm

1
2
3


C:\Users\nicho\Anaconda3\envs\data-x\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000000
1,1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000000
2,30,S0MA00109,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S4IN00014,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S4ME00055,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S6TN00216,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S6UT00063,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S6WY00068,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
2,30,S8MS00196,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000


In [98]:
df_committee_id.to_csv('committee_reformatted_test.csv')
df_committee_id

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,P00008052,C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000000
1,1,S8MN00438,C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000000
2,30,S8MS00196,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125000
3,34,H8OH05036,C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500000
4,35,H6NY25125,C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-25,DC,1.000000
5,36,S6OH00163,C00499921,S,Senate,[2012],J,Joint fundraising committee,T,2011-08-01,2012-01-24,2012-07-26,DC,0.250000
6,37,S8VA00214,C00522771,S,Senate,"[2012, 2014]",J,Joint fundraising committee,T,2012-06-04,2012-06-04,2013-02-26,VA,0.250000
7,38,S8VA00214,C00518597,S,Senate,[2012],J,Joint fundraising committee,T,2012-04-05,2012-06-11,2012-11-29,VA,0.250000
8,39,S8MS00055,C00543637,S,Senate,[2014],J,Joint fundraising committee,T,2013-04-12,2013-04-12,2013-10-29,VA,0.083333
9,40,S8MS00055,C00561845,S,Senate,[2014],J,Joint fundraising committee,T,2014-04-15,2014-04-15,2014-11-18,VA,0.052632


In [42]:
cand_row = df_candidate_id[df_candidate_id.candidate_id == 'S2NV00183']
cand_name = cand_row.name.values[0]
cand_name

'HELLER, DEAN'

In [53]:
cand_row = df_candidate_id[df_candidate_id.name == cand_name]
cand_id = cand_row.candidate_id.values
cand_row

,name,candidate_id,state,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
7298,"HELLER, DEAN",H6NV02164,NV,Challenger,REPUBLICAN PARTY,House,"[2006, 2008, 2010, 2012, 2014, 2016, 2018]","[2006, 2008, 2010, 2012]",2011-01-10,2013-04-26T09:04:30
7299,"HELLER, DEAN",S2NV00183,NV,Incumbent,REPUBLICAN PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-10-02,2018-10-03T01:11:26


In [46]:
df_comm[df_comm.candidate_ids == 'S2NV00183']

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
2,30,S2NV00183,C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
6,37,S2NV00183,C00522771,S,Senate,"[2012, 2014]",J,Joint fundraising committee,T,2012-06-04,2012-06-04,2013-02-26,VA,0.250


In [94]:
#Add df_candidate_id input to the previous function for candidate donor information collection function 

def get_committee_candidates(candidate_name, states, df_candidate_id, df_comm):
    cand_row = df_candidate_id[df_candidate_id.name == candidate_name]
    cand_ids = cand_row.candidate_id.values
    
    
    j = 0
    
    #Initialize dataframe
    cand_info = ['name', 'cand_id', 'state', 'cycle', 'incumbent', 'office_full', 'party']
    donation_levels = ['Donation Level 1',
               'Donation Level 2', 
               'Donation Level 3',
               'Donation Level 4',
               'Donation Level 5']
    levels = [0, 200, 500, 1000, 2000]
    columns = np.append(cand_info, states)
    columns = np.append(columns, donation_levels)

    df = pd.DataFrame(columns=columns)
    
    for cand_id in cand_ids:
        print(cand_id)
        comm_rows = df_comm[df_comm.candidate_ids == cand_id]
        
        #for row in comm_rows: 
        for index, row in comm_rows.iterrows(): #Iterate through each row of the dataframe comm_row
            candidate_donor_weight = row.candidate_donor_weight
            cycles = row.cycles
            comm_id = row.candidate_ids

            #Iterate through candidates cycles, add all cycles as new rows. Append to each column each
            print(cycles)
            for cycle in cycles:
                df_size = comm_donor_size(comm_id)
                df_state = comm_donor_state(comm_id)
                if df_size.values.size == 0 or df_state.values.size == 0:
                    continue

                donor_states = df_state.state_full

                #Build row of information (donor_states)

                # Idea
                # Create dictionary with states as keys and initialized with 0s as values. 
                initialize_array = np.zeros(len(states))
                state_dict = dict(zip(states, initialize_array))

                for ds in donor_states:            
                    # Overwrite select values with information from df_state
                    state_row = df_state[df_state.state_full == ds]
                    state_dict[ds] = state_row.total.values[0] * candidate_donor_weight


                # Add size features to state_dict 


                level_array = np.zeros(5)
                level_dict = dict(zip(donation_levels, level_array))

                for level in df_size['size']:
                    if level == 0:
                        donation_level = 'Donation Level 1'
                    if level == 200:
                        donation_level = 'Donation Level 2'
                    if level == 500:
                        donation_level = 'Donation Level 3'
                    if level == 1000:
                        donation_level = 'Donation Level 4'
                    if level == 2000:
                        donation_level = 'Donation Level 5'
                    size_row = df_size[df_size['size'] == level]
                    level_dict[donation_level] = size_row.total.values[0] * candidate_donor_weight


                cand_info_dict = {'name': candidate_name, 
                                  'cand_id': cand_id, 
                                  'state': row.state,
                                  'cycle': cycle,
                                  'incumbent': row.incumbent_challenge_full,
                                  'office_full': row.office_full,
                                  'party': row.party_full}
                state_dict = merge_two_dicts(cand_info_dict, state_dict)
                state_dict = merge_two_dicts(state_dict, level_dict)

                #return state_dict


                #return state_dict.values()
                df.loc[j] = list(state_dict.values()) # An array of the row information
                j += 1
    return df
    


In [95]:
get_committee_candidates('HELLER, DEAN', states, df_candidate_id, df_comm)
    

H6NV02164
S2NV00183
[2012]
[2012, 2014]


,name,cand_id,state,cycle,incumbent,office_full,party,Armed Forces Americas,Armed Forces Europe,Alaska,...,Washington,Wisconsin,West Virginia,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5


In [ ]:
df_committe_id

# Get Donor Information for Every Candidate in DataFrame A 

In [180]:
df_candidate_id.head()

,name,candidate_id,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,Open seat,CONSTITUTION PARTY,Senate,"[2012, 2014, 2016, 2018]","[2012, 2018]",2018-04-23,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,Challenger,REPUBLICAN PARTY,House,"[2012, 2014, 2016]",[2012],2012-02-22,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,Challenger,DEMOCRATIC PARTY,House,[2018],[2018],2017-04-26,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,Open seat,DEMOCRATIC PARTY,President,"[2006, 2008, 2010, 2012, 2014, 2016]",[2008],2007-03-13,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,Challenger,REPUBLICAN PARTY,House,"[2000, 2002, 2004]",[2000],2000-02-02,2002-04-12T00:00:00


In [252]:
cand_info = ['name', 'cand_id', 'state','cycle', 'incumbent', 'office_full', 'party']
donation_levels = ['Donation Level 1',
       'Donation Level 2', 
       'Donation Level 3',
       'Donation Level 4',
       'Donation Level 5']
columns = np.append(cand_info, states)
columns = np.append(columns, donation_levels)
df = pd.DataFrame(columns=columns)
i = 1
amount = len(df_candidate_id.name)
for name in df_candidate_id.name:
    df_name = candidate_donor_df(name, states)
    df = pd.concat([df, df_name], sort=False)
    print('{}/{}'.format(i, amount))
    i+=1
    if i == 10:
        break
df

[2012, 2014, 2016, 2018]
1/18617
[2012, 2014, 2016]
2/18617
[2018]
3/18617
[2006, 2008, 2010, 2012, 2014, 2016]
4/18617
[2000, 2002, 2004]
5/18617
[2006, 2008]
6/18617
[1992, 1994, 1996, 1998]
7/18617
[1990]
8/18617
[1994]
9/18617


,name,cand_id,cycle,Armed Forces Americas,Armed Forces Europe,Alaska,Alabama,Armed Forces Pacific,Arkansas,American Samoa,...,Wyoming,Foreign Countries,Donation Level 1,Donation Level 2,Donation Level 3,Donation Level 4,Donation Level 5,incumbent,office_full,party
0,"AALDERS, TIMOTHY NOEL",S2UT00229,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4415.00,250.00,2000.0,0.00,0.0,Open seat,Senate,CONSTITUTION PARTY
1,"AALDERS, TIMOTHY NOEL",S2UT00229,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4372.18,1000.00,0.0,1723.23,2400.0,Open seat,Senate,CONSTITUTION PARTY
0,"AANESTAD, SAMUEL",H2CA01110,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6936.76,8675.00,8500.0,18885.50,19500.0,Challenger,House,REPUBLICAN PARTY
0,"AARESTAD, DAVID",H8CO06237,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35291.90,22002.00,19375.0,12200.00,22800.0,Challenger,House,DEMOCRATIC PARTY
0,"ABAIR, PETER JON",H0MA01024,2000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4400.00,12324.00,23350.0,19000.00,0.0,Challenger,House,REPUBLICAN PARTY
1,"ABAIR, PETER JON",H0MA01024,2002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,500.0,5000.00,0.0,Challenger,House,REPUBLICAN PARTY
0,"ABATE, CAMILLE M",H6NJ05155,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,3600.00,2000.0,0.00,2100.0,Challenger,House,DEMOCRATIC PARTY
1,"ABATE, CAMILLE M",H6NJ05155,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,45195.58,28833.04,58290.0,63840.00,48400.0,Challenger,House,DEMOCRATIC PARTY
0,"ABATE, FRANK G",H2NJ12036,1992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,200.00,3600.00,3500.0,5000.00,0.0,Challenger,House,DEMOCRATIC PARTY
0,"ABBAS, JEFFREY LYN",H0IA03071,1990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,800.00,500.0,5000.00,0.0,Challenger,House,REPUBLICAN PARTY


In [ ]:
cand_info = ['name', 'cand_id', 'state','cycle', 'incumbent', 'office_full', 'party']
donation_levels = ['Donation Level 1',
       'Donation Level 2', 
       'Donation Level 3',
       'Donation Level 4',
       'Donation Level 5']
columns = np.append(cand_info, states)
columns = np.append(columns, donation_levels)
df = pd.DataFrame(columns=columns)


i = 1
amount = len(df_candidate_id.name)
for name in df_candidate_id.name:
    
    df_cand = candidate_donor_df(name, states, df_candidate_id)
    df_comm_cand = get_committee_candidates(name, states, df_candidate_id, df_committee_id)
    
    
    df = pd.concat([df, df_cand], sort=False)
    print('{}/{}'.format(i, amount))
    i+=1
    if i == 10:
        break
df

[2012, 2014, 2016, 2018]


NameError: name 'df' is not defined

## Retrive Basic Candidate Information by Candidate ID

In [ ]:
#Retrieve Basic Candidate Information
def retrieve_candidate_information(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/?sort=name&per_page=20&page=1&api_key=' + api_key
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)

#Retrieve Total/Financial Candidate Information
def retrieve_candidate_totals(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/totals/?api_key='+ api_key +'&sort=-cycle&per_page=20&page=1'
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)
    

## Find Nancy Pelosi Data 
Full name: Nancy Patricia D'Alesandro Pelosi

In [77]:
df_candidate_id.loc[df_candidate_id['name'].str.contains('PELOSI')]

,name,candidate_id,incumbent_challenge_full,party_full,office_full,cycles,election_years,last_file_date,load_date
12944,"PELOSI, NANCY",H8CA05035,Incumbent,DEMOCRATIC PARTY,House,"[1988, 1990, 1992, 1994, 1996, 1998, 2000, 200...","[1988, 1990, 1992, 1994, 1996, 1998, 2000, 200...",2016-11-16,2018-04-05T21:08:42


In [ ]:
retrieve_candidate_information('H8CA05035')

In [ ]:
retrieve_candidate_totals('H8CA05035')

## Retrieve Contributions by donor

### Construct Comittee with Candidate DataFrame